In [1]:
import re
import pandas as pd

import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher 
from spacy.tokens import Span 

import networkx as nx

import matplotlib.pyplot as plt
from tqdm import tqdm


from collections import Counter    

In [20]:
def clean_text(text):
    # if text.startswith("//"):
    #     return ""
    if "//" in text:
        return ""
    text = text.strip().lower()
    text = text.replace("•", "")
    text = " ".join(text.split(' '))
    return text

def match_patterns(text):
    patterns = [
        r"[A-Z].*?(is|are) called (a|an|the).*?\.", 
        r"\..*? defined as .*?\.",
        r"[Aa]n? [a-zA-Z]{0,20} [a-zA-Z]{0,20} [a-zA-Z]{0,20} is an? .*?\.",
        r"[Aa]n? [a-zA-Z]{0,20} [a-zA-Z]{0,20} is an? .*?\.",
        r"[Aa]n? [a-zA-Z]{0,20} is an? .*?\.",
        r"[Aa]n? [a-zA-Z]{0,20} is an? .*?\.",
        r"[[A-Z]{0,20} is an? .*?\."
                ]
    for pattern in patterns:
        if re.search(pattern, text):
            return True

def sentence_selection():
    texts = []

    df_caption = pd.read_csv("combined_captions_CS_241.csv", header=None, sep='\t')
    df_caption = df_caption.dropna()
    df_caption.columns = ['tid', 'course_date', 'seg_idx', 'start_time', 
    'end_time', 'caption', 'caption_lemmatized']

    for line in df_caption['caption']:
        # print(line)
        text = clean_text(line)
        if text != "":
            texts.append(text)

    matched_sentences = []
    matcher = Matcher(nlp.vocab)
    pattern = [{"POS": "DET", "OP":"?"}, {"POS": "NOUN", "OP": "+"}, {"POS": "AUX"}, {"POS": "DET"}]
    matcher.add("definition", [pattern])
    sources = [] # concept spans
    start_indexes = []
    
    matched_concepts = Counter()
    for line in tqdm(texts):
        if line[0].isdigit():
            continue
        sentences = [i.text for i in nlp(line).sents]
        for sent in sentences:
            if match_patterns(sent):
                # print(sent)
                s = nlp(sent)
                matches = matcher(s)
                if len(matches) == 0:
                    break
                noun_spans = []
                max_noun_span_len = 0
                for match_id, start, end in matches:
                    # string_id = nlp.vocab.strings[match_id]  # Get string representation
                    # span = s[start:end]  # The matched span
                    # print(span.text)
                    if end - start > max_noun_span_len:
                        max_noun_span_len = end - start
                        noun_span = s[start:end-2].text
                noun_spans.append(noun_span)
                matched_concepts[noun_span] += 1

                for pt in noun_spans:
                    start = sent.find(pt)
                    if start==-1:
                        continue
                    sources.append(noun_span)
                    matched_sentences.append(sent)
                    start_indexes.append(start)
                    break
    for k,v in matched_concepts.most_common():
        print(k,v)
    with open("extracted_sentences_caption.txt",'w') as f:
        f.write("sentence\tsource\tstart\n")
        for i in range(len(sources)):
            f.write(f"{matched_sentences[i]}\t{sources[i]}\t{start_indexes[i]}\n")



In [21]:
sentence_selection()

100%|██████████| 8537/8537 [05:29<00:00, 25.89it/s]

p 2
a pipe 1
memory 1
source 1
